In [1]:
from pyspark.sql import SparkSession
from pyspark import  SparkContext
import os
import pandas as pd

/tmp/ipykernel_387647/2081191832.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
spark = SparkSession.builder.master("yarn").getOrCreate()

/spark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/12 14:09:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/12 14:09:46 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
import pandas as pd

# Sample data for the Employee table
employee_data = {
    'id': [1, 2, 1, 2, 3, 1, 3, 1, 3, 1, 1],
    'month': [1, 1, 2, 2, 2, 3, 3, 4, 4, 7, 8],
    'salary': [20, 20, 30, 30, 40, 40, 60, 60, 70, 90, 90]
}

# Creating a pandas DataFrame for Employee table
employee_df = pd.DataFrame(employee_data)

# Displaying the Employee DataFrame
print("Employee table:")
print(employee_df)



df_person = spark.createDataFrame(employee_df)
df_person.createOrReplaceTempView("Employee")



Employee table:
    id  month  salary
0    1      1      20
1    2      1      20
2    1      2      30
3    2      2      30
4    3      2      40
5    1      3      40
6    3      3      60
7    1      4      60
8    3      4      70
9    1      7      90
10   1      8      90


In [24]:
query = """
    with add_2th_month as (
        select
            e.id,
            e.month,
            e.salary,
            ifnull(e1.salary, 0) as salary_1th,
            ifnull(e2.salary, 0) as salary_2th,
            e.salary + ifnull(e1.salary, 0) + ifnull(e2.salary, 0) as cumulative_salary 
        from Employee e
        left join Employee e1 on e.id = e1.id and (e.month - 1) = e1.month
        left join Employee e2 on e.id = e2.id and (e.month - 2) = e2.month
        where (e.id, e.month) not in (
            select
                id,
                max(month) as month
            from Employee 
            group by id
        )
    )
    
    select * from add_2th_month order by id asc, month desc
"""

In [25]:
spark.sql(query).show()

+---+-----+------+----------+----------+-----------------+
| id|month|salary|salary_1th|salary_2th|cumulative_salary|
+---+-----+------+----------+----------+-----------------+
|  1|    7|    90|         0|         0|               90|
|  1|    4|    60|        40|        30|              130|
|  1|    3|    40|        30|        20|               90|
|  1|    2|    30|        20|         0|               50|
|  1|    1|    20|         0|         0|               20|
|  2|    1|    20|         0|         0|               20|
|  3|    3|    60|        40|         0|              100|
|  3|    2|    40|         0|         0|               40|
+---+-----+------+----------+----------+-----------------+

